## Use case #1: using file inputParams.json (recommended use)

Execute the following cells only the first time running this notebook:

In [1]:
import os
os.chdir("..")

Loading the input parameters file in a new instance of the `controller`:

In [2]:
from synthDataGen.controller import DataControllerESIOS

controller = DataControllerESIOS("inputParams.json", "./synthDataGen/settings/")

Getting the first DataFrame. Then adjust it with the anual adjustments dictionary and resample it to the desired resolution:

In [3]:
df = controller.getDataFromSource()
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-17 18:00:00,41.61,75.44,49.75,49.92,56.51,69.14,70.35,65.84,80.94,48.88,60.25,70.05,58.10,41.30,100.63,217.92
2023-07-17 19:00:00,40.86,76.09,44.74,50.65,59.22,69.19,70.27,70.48,81.79,52.70,60.88,70.32,59.37,42.11,105.81,240.95
2023-07-17 20:00:00,41.05,78.51,43.54,51.17,64.49,69.25,70.58,71.02,81.73,54.21,60.27,70.35,58.92,42.13,105.59,251.37
2023-07-17 21:00:00,38.46,67.43,42.44,47.23,62.08,61.77,62.47,65.32,78.12,50.60,56.69,66.85,56.53,40.24,99.69,240.87
2023-07-17 22:00:00,34.16,61.92,39.69,47.38,62.34,60.20,62.31,65.02,76.21,48.84,56.14,63.96,58.04,36.50,95.68,207.98
2023-07-17 23:00:00,31.99,57.85,37.90,44.71,60.05,56.79,53.58,61.57,73.94,41.62,50.13,60.23,53.42,32.38,92.80,183.87
2023-07-18 00:00:00,29.69,53.33,35.51,40.17,54.39,52.53,43.55,54.16,69.04,39.43,47.37,58.25,48.97,29.80,85.46,187.92
2023-07-18 01:00:00,29.01,53.42,34.33,41.50,48.78,50.06,37.09,51.79,67.58,39.26,46.71,58.70,47.93,29.39,90.98,191.61
2023-07-18 02:00:00,27.73,52.82,32.82,37.52,49.00,49.01,36.54,50.75,63.42,38.71,46.35,58.05,47.96,28.79,91.24,191.94


In [4]:
df = controller.performAnualAdjustments(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-17 18:00:00,416.1,754.4,497.5,499.2,565.1,691.4,703.5,658.4,809.4,488.8,602.5,700.5,581.0,413.0,1006.3,2179.2
2023-07-17 19:00:00,408.6,760.9,447.4,506.5,592.2,691.9,702.7,704.8,817.9,527.0,608.8,703.2,593.7,421.1,1058.1,2409.5
2023-07-17 20:00:00,410.5,785.1,435.4,511.7,644.9,692.5,705.8,710.2,817.3,542.1,602.7,703.5,589.2,421.3,1055.9,2513.7
2023-07-17 21:00:00,384.6,674.3,424.4,472.3,620.8,617.7,624.7,653.2,781.2,506.0,566.9,668.5,565.3,402.4,996.9,2408.7
2023-07-17 22:00:00,341.6,619.2,396.9,473.8,623.4,602.0,623.1,650.2,762.1,488.4,561.4,639.6,580.4,365.0,956.8,2079.8
2023-07-17 23:00:00,319.9,578.5,379.0,447.1,600.5,567.9,535.8,615.7,739.4,416.2,501.3,602.3,534.2,323.8,928.0,1838.7
2023-07-18 00:00:00,296.9,533.3,355.1,401.7,543.9,525.3,435.5,541.6,690.4,394.3,473.7,582.5,489.7,298.0,854.6,1879.2
2023-07-18 01:00:00,290.1,534.2,343.3,415.0,487.8,500.6,370.9,517.9,675.8,392.6,467.1,587.0,479.3,293.9,909.8,1916.1
2023-07-18 02:00:00,277.3,528.2,328.2,375.2,490.0,490.1,365.4,507.5,634.2,387.1,463.5,580.5,479.6,287.9,912.4,1919.4


In [5]:
df = controller.downsample(df)
df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
datetime_utc,,,,,,,,,,,,,,,,
2023-07-17 18:00:00,412.35,757.65,472.45,502.85,578.65,691.65,703.10,681.60,813.65,507.90,605.65,701.85,587.35,417.05,1032.2,2294.35
2023-07-17 20:00:00,397.55,729.70,429.90,492.00,632.85,655.10,665.25,681.70,799.25,524.05,584.80,686.00,577.25,411.85,1026.4,2461.20
2023-07-17 22:00:00,330.75,598.85,387.95,460.45,611.95,584.95,579.45,632.95,750.75,452.30,531.35,620.95,557.30,344.40,942.4,1959.25
2023-07-18 00:00:00,293.50,533.75,349.20,408.35,515.85,512.95,403.20,529.75,683.10,393.45,470.40,584.75,484.50,295.95,882.2,1897.65
2023-07-18 02:00:00,280.20,534.85,320.80,376.75,514.90,493.70,397.35,511.40,619.50,390.05,461.45,583.30,486.05,288.10,911.7,1928.55


In [6]:

df = controller.upsample(df)

import pandas as pd
pd.set_option('display.precision', 2)

df

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
2023-07-17 18:00:00,412.35,757.65,472.45,502.85,578.65,691.65,703.10,681.60,813.65,507.90,605.65,701.85,587.35,417.05,1032.20,2294.35
2023-07-17 18:15:00,414.12,761.36,467.11,502.56,589.26,689.13,700.18,684.01,813.82,515.60,604.97,703.28,586.02,420.53,1036.60,2362.72
2023-07-17 18:30:00,414.85,763.02,461.78,501.97,598.78,686.03,696.75,685.74,813.42,521.68,603.74,703.74,584.72,422.83,1039.54,2417.52
2023-07-17 18:45:00,414.55,762.61,456.45,501.07,607.20,682.34,692.80,686.78,812.47,526.14,601.96,703.22,583.43,423.95,1041.02,2458.74
2023-07-17 19:00:00,413.22,760.15,451.13,499.87,614.53,678.06,688.33,687.14,810.95,528.97,599.63,701.73,582.16,423.90,1041.03,2486.38
2023-07-17 19:15:00,410.85,755.63,445.81,498.36,620.75,673.20,683.34,686.81,808.87,530.18,596.74,699.26,580.90,422.65,1039.57,2500.45
2023-07-17 19:30:00,407.45,749.04,440.50,496.55,625.88,667.75,677.83,685.79,806.23,529.76,593.31,695.81,579.67,420.23,1036.64,2500.94
2023-07-17 19:45:00,403.02,740.40,435.20,494.43,629.91,661.72,671.80,684.09,803.02,527.72,589.33,691.39,578.45,416.63,1032.25,2487.86
2023-07-17 20:00:00,397.55,729.70,429.90,492.00,632.85,655.10,665.25,681.70,799.25,524.05,584.80,686.00,577.25,411.85,1026.40,2461.20
2023-07-17 20:15:00,391.05,716.94,424.61,489.27,634.69,647.90,658.18,678.63,794.92,518.76,579.72,679.63,576.07,405.89,1019.08,2420.96


# Use case #2: passing parameters


In [7]:
from datetime import datetime

d = datetime(2023, 6, 5, 7, 0)

df = controller.getDataFromSource(initialYear=2018, initDatetime=d, hoursAhead=6)
df

,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,67.83,53.13,37.07,81.00,193.53
2023-06-05 07:00:00,69.11,50.74,37.06,81.43,177.57
2023-06-05 08:00:00,69.87,50.44,35.33,85.35,174.79
2023-06-05 09:00:00,69.09,50.50,34.96,85.31,177.76
2023-06-05 10:00:00,68.72,51.26,35.24,85.29,177.64
2023-06-05 11:00:00,69.03,49.11,35.13,85.36,176.71
2023-06-05 12:00:00,69.02,47.22,32.09,83.26,173.96


In [8]:
df = controller.performAnualAdjustments(df, adjustmentsDict={2018: 1.012, 2019: 1.023, 2020: 1.0145, 2021: 1.03, 2022: 1.08})
df

,2018,2019,2020,2021,2022
datetime_utc,,,,,
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98
2023-06-05 10:00:00,69.54,52.44,35.75,87.85,191.85
2023-06-05 11:00:00,69.86,50.24,35.64,87.92,190.85
2023-06-05 12:00:00,69.85,48.31,32.56,85.76,187.88


In [9]:
df = controller.upsample(df, frequency="20T", method="spline", order=3)

import pandas as pd
pd.set_option('display.precision', 2)

df

,2018,2019,2020,2021,2022
2023-06-05 06:00:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 06:20:00,69.25,53.31,37.41,83.74,201.75
2023-06-05 06:40:00,69.70,52.60,36.97,84.39,196.71
2023-06-05 07:00:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 07:20:00,70.24,51.79,36.42,85.63,190.74
2023-06-05 07:40:00,70.36,51.62,36.27,86.21,189.39
2023-06-05 08:00:00,70.71,51.60,35.84,87.91,188.77
2023-06-05 08:20:00,70.36,51.59,36.12,87.20,188.97
2023-06-05 08:40:00,70.28,51.67,36.09,87.61,189.48
2023-06-05 09:00:00,69.92,51.66,35.47,87.87,191.98


In [15]:
df = controller.downsample(df, frequency="22.73T", aggregationFunc="mean")
df

,2018,2019,2020,2021,2022
2023-06-05 05:22:00,68.64,54.35,37.61,83.43,209.01
2023-06-05 05:45:00,69.25,53.31,37.41,83.74,201.75
2023-06-05 06:08:00,69.70,52.60,36.97,84.39,196.71
2023-06-05 06:31:00,69.94,51.91,37.60,83.87,191.78
2023-06-05 06:54:00,70.24,51.79,36.42,85.63,190.74
2023-06-05 07:17:00,70.36,51.62,36.27,86.21,189.39
2023-06-05 07:40:00,70.53,51.60,35.98,87.56,188.87
2023-06-05 08:03:00,70.28,51.67,36.09,87.61,189.48
2023-06-05 08:26:00,69.92,51.66,35.47,87.87,191.98
2023-06-05 08:49:00,70.02,51.84,36.01,88.18,190.97
